In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    for angle in range(-180,180,1):
    
        df1 = df.loc[(df.phi_deg>(angle-1.0)) & (df.phi_deg<(angle+1.0))]
    
        x  = df1.x.values
        y  = df1.y.values
        z  = df1.z.values
        r  = np.sqrt(x**2+y**2)
        d  = np.sqrt(x**2+y**2+z**2)
        a  = np.arctan2(y,x)

        x2 = df1['x']/d
        y2 = df1['y']/d
        z2 = df1['z']/r
    
        r2 = np.sqrt(x2**2 + y2**2)
        theta= np.arctan2(r, z)
        theta1 = np.arctan2(r2, z2)

    
        tan_dip = phi/theta
        tan_dip1 = phi/z2
        z2_1 = 1/z2
        z2_2 = phi/z2 + 1/z2

        dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
        dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
        scores = []

        a_start,a_step,a_num = 0.00100,0.0000095,150
    
        params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

        if 1: 
            pool = Pool(processes=1)
            ls   = pool.map( one_loop1, params )


        if 0:
            ls = [ one_loop(param) for param in  params ]


        ##------------------------------------------------

        num_hits=len(df)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [108]:
def trackML(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    print(Niter)
    Niter = Niter.astype(np.int32)
    print(Niter)
    print(w1, w2, w3, Niter)
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [119]:
def trackML2(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [109]:
######################## 
### working function ###
# ########################
# trackML <- function(dfh,w1,w2,w3,Niter){
#   dfh[,s1:=hit_id]
#   dfh[,N1:=1L] 
#   dfh[,r:=sqrt(x*x+y*y+z*z)]
#   dfh[,rt:=sqrt(x*x+y*y)]
#   dfh[,a0:=atan2(y,x)]
#   dfh[,z1:=z/rt]
#   dfh[,z2:=z/r]
#   mm     <-  1
#   for (ii in 0:Niter) {
#     mm <- mm*(-1)
#     dfh[,a1:=a0+mm*(rt+0.000005*rt^2)/1000*(ii/2)/180*pi]
#     dfh[,sina1:=sin(a1)]
#     dfh[,cosa1:=cos(a1)]
#     dfs=scale(dfh[,.(sina1,cosa1,z1,z2)])
# 	cx <- c(w1,w1,w2,w3)
#     for (jj in 1:ncol(dfs)) dfs[,jj] <- dfs[,jj]*cx[jj]
#     res=dbscan(dfs,eps=0.0035,minPts = 1)
#     dfh[,s2:=res$cluster]
#     dfh[,N2:=.N, by=s2]
#     maxs1 <- max(dfh$s1)
#     dfh[,s1:=ifelse(N2>N1 & N2<20,s2+maxs1,s1)]
#     dfh[,s1:=as.integer(as.factor(s1))]
#     dfh[,N1:=.N, by=s1]    
#   }
#   return(dfh$s1)
# }

In [110]:
def Fun4BO(w1,w2,w3,Niter):
    track_id = trackML(hits, w1,w2,w3,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(2):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return list({'Score':sc, 'Pred':0})

In [111]:
#######################################
# function for Bayessian Optimization #
#   (needs lists: Score and Pred)     #
#######################################
# Fun4BO <- function(w1,w2,w3,Niter) { 
#    dfh$s1 <- trackML(dfh,w1,w2,w3,Niter)
#    sub=data.table(event_id=nev,hit_id=dfh$hit_id,track_id=dfh$s1)
#    sc <- score(sub,dft)
#    list(Score=sc,Pred=0)
# }

In [112]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576, eps = 0.0,verbose = True)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
174.91171047135347
174
1.0307915160224026 0.6665432240143782 0.23273448176797165 174
 179.0000000[ 0] score : 0.53242166
 179.0000000[ 1] score : 0.53706344
--------------------------------------
0.5347425490280032


ValueError: setting an array element with a sequence.

In [122]:
w1 = 1.0307915160224026
w2 = 0.6665432240143782
w3 = 0.23273448176797165
Niter = 174

In [123]:
def run_make_submission():

    data_dir = '../data/test'


    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)

            track_id = trackML2(hits, w1, w2, w3, Niter)
            #track_id = back_fit(track_id,hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, track_id))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub2/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub2/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub2/submission_dbscan2.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [124]:
run_make_submission()

event_id : 000000000  , 3 min
event_id : 000000001  , 8 min
event_id : 000000002  , 13 min
event_id : 000000003  , 18 min
event_id : 000000004  , 23 min
event_id : 000000005  , 28 min
event_id : 000000006  , 33 min
event_id : 000000007  , 38 min
event_id : 000000008  , 44 min
event_id : 000000009  , 49 min
event_id : 000000010  , 54 min
event_id : 000000011  , 59 min
event_id : 000000012  , 64 min
event_id : 000000013  , 69 min
event_id : 000000014  , 75 min
event_id : 000000015  , 79 min
event_id : 000000016  , 83 min
event_id : 000000017  , 89 min
event_id : 000000018  , 94 min
event_id : 000000019  , 99 min
event_id : 000000020  , 103 min
event_id : 000000021  , 108 min
event_id : 000000022  , 113 min
event_id : 000000023  , 119 min
event_id : 000000024  , 124 min
event_id : 000000025  , 130 min
event_id : 000000026  , 134 min
event_id : 000000027  , 139 min
event_id : 000000028  , 144 min
event_id : 000000029  , 148 min
event_id : 000000030  , 153 min
event_id : 000000031  , 157 mi